In [2]:
import requests, bs4
from bs4 import BeautifulSoup as bs
import pandas as pd

Step 1: Loop through basketball-reference for per game data from 2018 - 2021

In [436]:
years = ['2018','2019','2020','2021']

for i in years:
    
    #Extract URL for web-scraping
    url=f'https://www.basketball-reference.com/leagues/NBA_{i}_per_game.html'
    response = requests.get(url)
    status = response.status_code
    if status == 200:
      page = response.text
      soup = bs(page)
    else:
      print(f"Oops! Received status code {status}")
    
    #Generate column titles
    headers = [th.getText() for th in soup.find('tr').findAll('th')]
    headers.pop(0)
    
    #Get box score data
    players = soup.find_all('tr')[1:]
    stats_list = [[td.text for td in players[j].find_all('td')] for j in range(len(players))]
    
    #Convert to Pandas, export to csv
    nba_box = pd.DataFrame(stats_list, columns = headers)
    nba_box['YEAR'] = int(i)
    nba_box.dropna(inplace=True)
    nba_box.to_csv(f'data/nba_box_{i}.csv',index=False)


Step 2: basketball-reference per 100 possessions from 2018 - 2021

In [437]:
years = ['2018','2019','2020','2021']

for i in years:
    
    #Extract URL for web-scraping
    url=f'https://www.basketball-reference.com/leagues/NBA_{i}_per_poss.html'
    response = requests.get(url)
    status = response.status_code
    if status == 200:
      page = response.text
      soup = bs(page)
    else:
      print(f"Oops! Received status code {status}")
    
    #Generate column titles
    headers = [th.getText() for th in soup.find('tr').findAll('th')]
    headers.pop(0)
    
    #Get box score data
    players = soup.find_all('tr')[1:]
    stats_list = [[td.text for td in players[j].find_all('td')] for j in range(len(players))]
    
    #Convert to Pandas, export to csv
    nba_per100 = pd.DataFrame(stats_list, columns = headers)
    nba_per100['YEAR'] = int(i)
    nba_per100.dropna(inplace=True)
    nba_per100.drop(columns='',inplace=True)
    nba_per100.to_csv(f'data/nba_per100_{i}.csv',index=False)

Step 3: ESPN Salary Data

In [438]:
years = ['2018','2019','2020','2021']
for h in years:
    master_salary_list = []
    for i in range(1,16):

        #Extract URL for web-scraping
        url=f'http://www.espn.com/nba/salaries/_/year/{h}/page/{i}/seasontype/4'
        response = requests.get(url)
        status = response.status_code
        if status == 200:
          page = response.text
          soup = bs(page)
        else:
          print(f"Oops! Received status code {status}")

        #Generate headers
        headers = [td.getText() for td in soup.find('table').find_all('tr')[0].findAll('td')]

        #Generate salary list for respective page
        x = soup.find('table').find_all('tr')
        salary_list = [[td.text for td in x[j].find_all('td')] for j in range(len(x))]
        master_salary_list += salary_list
    
    #Convert to dataframe and conduct initial cleaning
    salarydf = pd.DataFrame(master_salary_list, columns = headers)
    salarydf['SALARY'] = salarydf['SALARY'].str.replace('$','')
    salarydf['SALARY'] = salarydf['SALARY'].str.replace(',','')
    salarydf['NAME'] = salarydf['NAME'].str.split(',').str[0]
    salarydf = salarydf[salarydf['RK'] != 'RK']
    salarydf['YEAR'] = h
    salarydf.to_csv(f'data/nba_salary_{h}.csv',index=False)

/var/folders/75/fwf63z_j1jvf891tg0n68jn80000gn/T/ipykernel_35641/539352140.py:24: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  salarydf['SALARY'] = salarydf['SALARY'].str.replace('$','')
